# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
data = get_date_list(data)
data = read_data(data)

Record num : 39426


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
        # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------


    # lema before after loops --------------------------------
    if len(data['lema_list']) < data['lema_len']:
        data = before_lema(data)        
        continue

    if len(data['lema_list']) == data['lema_len']:
        data = after_lema(data)                  
    # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    
    data = get_position(data)
    if not data['position']:
        continue
        
    # Get Dirs --------------------------------
    if len(data['dir_list']) < 2:
        data['dir_list'].append(data['position'])   
        continue

    elif len(data['dir_list']) == 2:
        data = get_cross_dir(data)
    # ----------------------------------------------------------  
    
    data = get_trend_lines_back_line(data)
#     data = get_trend_lines_moving(data)
    data = trend_angle_check(data)

100%|████████████████████████████████████| 39426/39426 [06:51<00:00, 95.71it/s]


In [4]:
data['df']

,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h_trend_calc_spot,l_trend_calc_spot,h,l,h_line,l_line
0,20210106 00:00:00.067,1.22985,1.22990,1.229875,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,4.018402,11.640010,NaN,NaN,NaN,NaN,NaN,1.231334,1.229177
1,20210106 00:00:00.169,1.22987,1.22992,1.229895,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,-3.313845,1.822470,NaN,NaN,NaN,NaN,NaN,1.231659,1.229678
2,20210106 00:00:00.270,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,-3.313845,1.822470,NaN,NaN,NaN,NaN,NaN,1.231659,1.229678
3,20210106 00:00:00.372,1.22989,1.22994,1.229915,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,-3.313845,1.822470,NaN,NaN,NaN,NaN,NaN,1.231659,1.229678
4,20210106 00:00:00.980,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,-3.313845,1.822470,NaN,NaN,NaN,NaN,NaN,1.231659,1.229678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39421,20210106 07:07:07.911,1.23206,1.23210,1.232080,2021-01-06 07:07:07,1.23204,1.231985,up,,NaN,18.524776,14.748173,up,1.232470,1.231154,NaN,NaN,1.232470,1.231154
39422,20210106 07:07:08.013,1.23206,1.23209,1.232075,2021-01-06 07:07:08,1.23204,1.231985,up,,NaN,18.524776,14.748173,up,1.232470,1.231154,NaN,NaN,1.232470,1.231154
39423,20210106 07:07:08.165,1.23205,1.23209,1.232070,2021-01-06 07:07:08,1.23204,1.231985,up,,NaN,18.524776,14.748173,up,1.232470,1.231155,NaN,NaN,1.232470,1.231155
39424,20210106 07:07:08.267,1.23203,1.23207,1.232050,2021-01-06 07:07:08,1.23204,1.231985,up,,NaN,18.524776,14.748173,up,1.232471,1.231155,NaN,NaN,1.232471,1.231155


In [5]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

Records   : 39426
Hours     : 7.12
Recs/hour : 5537
Recs/min  : 92
Recs/sec  : 1.5


,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h_trend_calc_spot,l_trend_calc_spot,h,l,h_line,l_line
0,20210106 00:00:00.067,1.22985,1.22990,1.229875,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,4.018402,11.64001,NaN,NaN,NaN,NaN,NaN,1.231334,1.229177
1,20210106 00:00:00.169,1.22987,1.22992,1.229895,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,-3.313845,1.82247,NaN,NaN,NaN,NaN,NaN,1.231659,1.229678
2,20210106 00:00:00.270,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,-3.313845,1.82247,NaN,NaN,NaN,NaN,NaN,1.231659,1.229678
3,20210106 00:00:00.372,1.22989,1.22994,1.229915,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,-3.313845,1.82247,NaN,NaN,NaN,NaN,NaN,1.231659,1.229678
4,20210106 00:00:00.980,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,-3.313845,1.82247,NaN,NaN,NaN,NaN,NaN,1.231659,1.229678


In [6]:
plot_graph(data)

In [7]:
data['df'].to_csv('data/temp.csv')

In [8]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)